In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, weights=[0.9, 0.1], flip_y=0, random_state=42)

classes, counts = np.unique(y, return_counts=True)
total = counts.sum()
percentages = (counts / total) * 100

# Imprimir resultados
print(f"Clase 0: {percentages[0]:.1f}% ({counts[0]} muestras)")
print(f"Clase 1: {percentages[1]:.1f}% ({counts[1]} muestras)")

Clase 0: 90.0% (900 muestras)
Clase 1: 10.0% (100 muestras)


In [3]:
# Split the dataset into training (70%) and testing (30%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [4]:
# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

In [5]:
# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       270
           1       0.62      0.50      0.56        30

    accuracy                           0.92       300
   macro avg       0.79      0.73      0.76       300
weighted avg       0.91      0.92      0.92       300



In [6]:
report_dict = classification_report(y_test, y_pred, output_dict=True) 
# output_dict=True para obtener un diccionario con los resultados:
report_dict

{'0': {'precision': 0.9456521739130435,
  'recall': 0.9666666666666667,
  'f1-score': 0.9560439560439561,
  'support': 270.0},
 '1': {'precision': 0.625,
  'recall': 0.5,
  'f1-score': 0.5555555555555556,
  'support': 30.0},
 'accuracy': 0.92,
 'macro avg': {'precision': 0.7853260869565217,
  'recall': 0.7333333333333334,
  'f1-score': 0.7557997557997558,
  'support': 300.0},
 'weighted avg': {'precision': 0.9135869565217392,
  'recall': 0.92,
  'f1-score': 0.9159951159951161,
  'support': 300.0}}

➡️ Vamos a usar el `report_dict` (que genera `classification_report` de `sklearn`) para registrar las métricas en MLflow.

📌 **Mejor práctica:** entrenamos el modelo, obtenemos todas las métricas que necesitamos, y recién ahí las registramos en MLflow.

Esto nos permite tener todo centralizado y ordenado: parámetros, métricas y el modelo entrenado.

In [7]:
import mlflow
from mlflow.models.signature import infer_signature

In [9]:
mlflow.set_experiment("First Experiment")
mlflow.set_tracking_uri(uri="http://localhost:5000/") # Es el uri que obtuvimos cuando corrimos mlflow ui

with mlflow.start_run(run_name="Baseline_LogisticRegression"):
    # Log de hiperparámetros del modelo
    mlflow.log_params(params) 
    # Log de métricas de evaluación
    mlflow.log_metrics({
        'accuracy': report_dict['accuracy'],
        'recall_class_0': report_dict['0']['recall'],
        'recall_class_1': report_dict['1']['recall'],
        'precision_class_0': report_dict['0']['precision'],
        'precision_class_1': report_dict['1']['precision'],
        'f1_score_macro': report_dict['macro avg']['f1-score']
    })

    # input_example: muestra representativa de las entradas que espera el modelo
    # Sirve para documentar el modelo y facilitar su uso o despliegue
    input_example = pd.DataFrame(X_test[:2], columns=[f"feature_{i}" for i in range(X.shape[1])])
    
    # signature: describe la forma y tipo de las entradas y salidas del modelo
    # Se infiere automáticamente a partir de X_test e y_pred
    signature = infer_signature(X_test, y_pred)
    
    # Log del modelo
    mlflow.sklearn.log_model(lr, 
                             "Logistic Regression", 
                             input_example = input_example, 
                             signature = signature)  

🏃 View run Baseline_LogisticRegression at: http://localhost:5000/#/experiments/744018259773384787/runs/4d6b1e7af7d04e31b9dc975563d59584
🧪 View experiment at: http://localhost:5000/#/experiments/744018259773384787


✅¡Listo! Tu experimento y tu run se registraron correctamente en MLflow 🎉

Como ya tenemos todo registrado en MLflow —el modelo, los parámetros, las métricas, el input esperado y la signature—, **nuestro experimento está empaquetado como una "unidad atómica"**.

💡 Esto significa que ahora podemos:

- Servir el modelo como una API REST con un solo comando
- Exportarlo para producción
- Reproducir exactamente cómo fue entrenado
- 🐳 Dockerizarlo fácilmente

Todo esto es posible porque ahora nuestro modelo es una unidad autocontenida que MLflow puede entender, mover y ejecutar sin que tengamos que reescribir código o recordar configuraciones.

